# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 观测结果预处理

---
*@author: Evan*\
*@date: 2023-07-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
xls=pd.read_excel('D:/data/Project_Anqing/202306/obsdata/originaldata.xlsx',header=0,na_values=['—',-99])

xls['datetime'] = pd.to_datetime(xls['时间'].astype(str))

xls.drop(['城市','市县','时间'], axis=1, inplace=True)

xls.set_index('datetime',inplace=True)
xls

,站点,SO2,NO2,NOx,NO,CO,O3,PM10,PM2.5,风速,风向,气压,气温,湿度,降水量,能见度
datetime,,,,,,,,,,,,,,,,
2023-06-01 00:00:00,安庆大学,8,7,8,0,0.586,84,38,39,0.5,6.5,1002.6,23.7,85,0.000(BB),NaN
2023-06-01 01:00:00,安庆大学,8,8,8,0,0.636,77,37,41,0.4,346.2,1002,23.6,86,0.000(BB),NaN
2023-06-01 02:00:00,安庆大学,7,6,7,0,0.587,80,46,37,1,2.9,1001.6,23.4,88,0.000(BB),NaN
2023-06-01 03:00:00,安庆大学,8,8,8,1,0.571,63,43,34,0.5,304.2,1001,23,92,0.000(BB),NaN
2023-06-01 04:00:00,安庆大学,7,8,9,0,0.558,57,40,32,0.8,2.5,1000.9,22.8,93,0.000(BB),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 19:00:00,高新区,-2,13,17,2,0.593,59,15,6,1.1,193.2,995.2,25.3,100,NaN,NaN
2023-06-30 20:00:00,高新区,-2,11,14,2,0.538,62,12,11,1,263.5,995.6,25.1,100,NaN,NaN
2023-06-30 21:00:00,高新区,-2,9,12,2,0.542,66,16,12,1,272,996,25,100,NaN,NaN


In [3]:
# 将所有因包含括号等字符被读取为字符串的数据设置为空值
xls = xls.replace(to_replace=[r'\(.*?\)'], value=np.nan, regex=True)

In [4]:
xls[xls['站点']=='安庆政务中心']['气压']

datetime
2023-06-01 00:00:00    100.5
2023-06-01 01:00:00    100.4
2023-06-01 02:00:00    100.4
2023-06-01 03:00:00    100.3
2023-06-01 04:00:00    100.3
                       ...  
2023-06-30 19:00:00    100.0
2023-06-30 20:00:00    100.1
2023-06-30 21:00:00    100.1
2023-06-30 22:00:00    100.1
2023-06-30 23:00:00    100.2
Name: 气压, Length: 720, dtype: float64

In [5]:
# 发现安庆政务中心站点的气压数据整体小一个量级
data_replace=xls[xls['站点'].isin(['安庆政务中心'])]['气压']
xls['气压'].replace(data_replace.values,data_replace.values*10,inplace=True)

In [6]:
# 根据站点分组导出
grouped = xls.groupby('站点')
t_index=pd.date_range('2023-06-01T00','2023-06-30T23',freq='h')

for group_name, group_df in grouped:
    # 时间序列如有缺失，则补充空值占位
    group_df.reindex(t_index)
    group_df.to_excel(f'D:/Download/{group_name}.xlsx', index=True)

In [7]:
# 将全部站点取平均并导出
xls.groupby(xls.index).mean().to_excel('D:/Download/allsite.xlsx',index=True)

In [8]:
# 将城区站与通道站分开
path = 'D:/data/Project_Anqing/202306/obsdata/'

urban_path=path + 'urban/'
other_path=path + 'others/'
urban_files = [os.path.splitext(filename)[0] for filename in os.listdir(urban_path) if filename.endswith('.xlsx')]
other_files = [os.path.splitext(filename)[0] for filename in os.listdir(other_path) if filename.endswith('.xlsx')]

urban=xls[xls['站点'].isin(urban_files)]
other=xls[xls['站点'].isin(other_files)]

# 平均并导出
urban.groupby(urban.index).mean().to_excel('D:/Download/urban.xlsx',index=True)
other.groupby(other.index).mean().to_excel('D:/Download/others.xlsx',index=True)